In [4]:
import gradio as gr
import random
import time
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
import os
import re
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
from dotenv import load_dotenv
load_dotenv("en.env")
from langchain.utilities import BingSearchAPIWrapper
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from gradio_tools import (StableDiffusionTool, ImageCaptioningTool, StableDiffusionPromptGeneratorTool,
                          TextToVideoTool)


BASE_URL = os.environ.get('OPENAI_API_BASE')
API_KEY = os.environ.get('OPENAI_API_KEY')
Version = os.environ.get('OPENAI_API_VERSION')
DEPLOYMENT_NAME = "chatgpt0301"




with gr.Blocks(css=customCSS) as demo:
    chatbot = gr.Chatbot().style(height="100%")
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    search = BingSearchAPIWrapper()
    tools = [StableDiffusionTool().langchain,StableDiffusionPromptGeneratorTool().langchain]
    '''
    tools = [
        Tool(
            name = "Current Search",
            func=search.run,
            description="useful for when you need to answer questions about current events or the current state of the world"
        ),
    ]
    '''
    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        llm = AzureChatOpenAI(
            temperature=0,
            model_name="gpt-35-turbo",
            openai_api_base=BASE_URL,
            openai_api_version="2023-03-15-preview",
            deployment_name="chatgpt0301",
            openai_api_key=API_KEY,
            openai_api_type = "azure",
            
        )
        '''
        llm=AzureOpenAI(temperature=0,deployment_name="davinci003", model_name="text-davinci-003")
        '''
        #llm=OpenAI(temperature=0)
        memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        for i in history[0:-1]:
            memory.save_context({"input": i[0]}, {"output": ''.join(i[1])})
        #tools = [StableDiffusionTool().langchain, ImageCaptioningTool().langchain,StableDiffusionPromptGeneratorTool().langchain, TextToVideoTool().langchain]
        


        agent = initialize_agent(tools, llm, memory=memory, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True)
        bot_message = agent.run(history[-1][0])
        '''
        "Please create a photo of a dog riding a skateboard "
                                  "but improve my prompt prior to using an image generator."
        '''
        #bot_message=agent_chain.run(history[-1][0])
        #print(bot_message)
        history[-1][1] = ""
        
        isPic = re.search(r'(/[^"]+png)', str(bot_message))
        if isPic:
            print(isPic.group(1))
            history[-1][1] = (isPic.group(1)," "+bot_message)
            yield history
        else:
            for character in bot_message:
                history[-1][1] += character
                #time.sleep(0.05)
                yield history

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.queue()
demo.launch()


Loaded as API: https://gradio-client-demos-text-to-image.hf.space ✔
Loaded as API: https://microsoft-promptist.hf.space ✔
Running on local URL:  http://127.0.0.1:7919

To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "Hello! How can I assist you today?"
}

> Finished chain.
